In [1]:
import zarr
import xarray as xr
import numpy as np
import numpy as np
import dask
from pyseq import image_analysis as ia
from dask.distributed import Client
import torch
import joblib
from dask_jobqueue import SLURMCluster
import skimage
import time
import pickle
from os.path import exists, join
from joblib import Parallel, delayed
from joblib import parallel_backend
from dask.distributed import progress
from pathlib import Path

/gpfs/commons/home/jsingh/.conda/envs/spatial/lib/python3.9/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


In [1]:
class snakemake():
    input  = ['/gpfs/commons/home/jsingh/results/20210323_4i4color/final_zarr/m1a.zarr', 
              '/gpfs/commons/home/jsingh/results/20210323_4i4color/masks/m1a.tiff']
    output = ['/gpfs/commons/home/jsingh/results/m1a.pkl']

In [3]:
labels = skimage.io.imread(snakemake.input[1])
image_path = Path(snakemake.input[0])
im_name = image_path.stem
image = xr.open_zarr(image_path).to_array()
image = image.squeeze().drop_vars('variable').rename(im_name)

In [4]:
marker_list = ['LMN1b', 'GFAP','ELAVL2','MBP','PVALB']

In [5]:
plane_dict = {}
for mark in marker_list:
    try:
        plane_dict.update({mark: image.sel(marker = mark)})
    except:
        pass

In [6]:
plane_dict

{'LMN1b': <xarray.DataArray 'm1a' (row: 1830, col: 2044)>
 dask.array<getitem, shape=(1830, 2044), dtype=float64, chunksize=(1830, 2044), chunktype=numpy.ndarray>
 Coordinates:
     channel   int64 dask.array<chunksize=(), meta=np.ndarray>
   * col       (col) int64 0 1 2 3 4 5 6 7 ... 2037 2038 2039 2040 2041 2042 2043
     cycle     int64 dask.array<chunksize=(), meta=np.ndarray>
     marker    <U8 'LMN1b'
     obj_step  int64 ...
   * row       (row) int64 0 1 2 3 4 5 6 7 ... 1823 1824 1825 1826 1827 1828 1829,
 'GFAP': <xarray.DataArray 'm1a' (row: 1830, col: 2044)>
 dask.array<getitem, shape=(1830, 2044), dtype=float64, chunksize=(1830, 2044), chunktype=numpy.ndarray>
 Coordinates:
     channel   int64 dask.array<chunksize=(), meta=np.ndarray>
   * col       (col) int64 0 1 2 3 4 5 6 7 ... 2037 2038 2039 2040 2041 2042 2043
     cycle     int64 dask.array<chunksize=(), meta=np.ndarray>
     marker    <U8 'GFAP'
     obj_step  int64 ...
   * row       (row) int64 0 1 2 3 4 5 6 7 ..

In [7]:
############  CPU PART #################

In [8]:
if torch.cuda.is_available() == False:

    def get_cluster(queue_name = 'pe2', log_dir=None):
        """ Make dask cluster w/ workers = 2 cores, 32 G mem, and 1 hr wall time.

            return cluster, client
        """

        cluster = SLURMCluster(
                    queue = queue_name, 
                    cores = 6 ,
                    memory = '60G',
                    walltime='1:00:00')
                    #extra=["--lifetime", "55m", "--lifetime-stagger", "4m"])
        client = Client(cluster, timeout="50s")

        return cluster, client

    cluster, client = get_cluster()

    def scale_cluster(count): 
        cluster.scale(count)
        return cluster.dashboard_link
    scale_cluster(5)

    #Way 3: Computing the same using dask compute without persisting Data
    val = np.max(labels)
    
    
    def get_pixels(lab,pl):
        m = plane_dict[pl].values[labels == lab+1].mean()
        return m
    
    mean_intensity_per_marker = {}
    for plane in plane_dict.keys():
        
        print(plane)
    
        with parallel_backend('dask',scheduler_host=cluster.scheduler._address,wait_for_workers_timeout=20):
            mean_int = Parallel(n_jobs=-1)(delayed(get_pixels)(lab, pl = plane) for lab in range(val))
        mean_intensity_per_marker.update({plane:mean_int})
    
    #intensity_dict = {}
    #intensity_dict.update({'m1a':results})
    with open(snakemake.output[0], 'wb') as f:
        pickle.dump(mean_intensity_per_marker, f)
        
    client.close()
    cluster.close()
        
    
else:
    
    def get_mean_intensity(pl):
        result_ar = np.zeros(mx)
        tr = torch.from_numpy(pl)
        for r in range(mx):
            result_ar[r] = (tr[lab == r+1]).float().mean()
        return result_ar

    lab = torch.from_numpy(labels.astype('int'))
    mx = np.max(labels)
    
    mean_intensity_per_marker = {}
    for plane in plane_dict.keys():
        pl = plane_dict[plane].values
        mean_int = get_mean_intensity(pl)
        mean_intensity_per_marker.update({plane:mean_int})
    
    with open(snakemake.output[0], 'wb') as f:
        pickle.dump(mean_intensity_per_marker, f)
    
        

    

LMN1b
GFAP
ELAVL2
MBP


In [2]:
import zarr
import xarray as xr
import numpy as np
import numpy as np
import dask
from pyseq import image_analysis as ia
from dask.distributed import Client
import torch
import joblib
from dask_jobqueue import SLURMCluster
import skimage
import time
import pickle
from os.path import exists, join
from joblib import Parallel, delayed
from joblib import parallel_backend
from dask.distributed import progress
from pathlib import Path


labels = skimage.io.imread(Path(snakemake.input[1]))
image_path = Path(snakemake.input[0])
im_name = image_path.stem
image = xr.open_zarr(image_path).to_array()
image = image.squeeze().drop_vars('variable').rename(im_name)

marker_list = ['LMN1b','GFAP','ELAVL2','MBP','PVALB','IBA1','PDGFRA','MAP2','NFH']

plane_dict = {}

for mark in marker_list:
    try:
        plane_dict.update({mark: image.sel(marker = mark)})
    except:
        pass
    
if torch.cuda.is_available() == False:

    def get_cluster(queue_name = 'pe2', log_dir=None):
        """ Make dask cluster w/ workers = 2 cores, 32 G mem, and 1 hr wall time.

            return cluster, client
        """

        cluster = SLURMCluster(
                    queue = queue_name, 
                    cores = 6 ,
                    memory = '60G',
                    walltime='1:00:00')
                    
        client = Client(cluster, timeout="50s")

        return cluster, client

    cluster, client = get_cluster()

    def scale_cluster(count): 
        cluster.scale(count)
        return cluster.dashboard_link
    scale_cluster(5)

    
    val = np.max(labels)
    
    
    def get_pixels(lab,pl):
        m = plane_dict[pl].values[labels == lab+1].mean()
        return m
    
    mean_intensity_per_marker = {}
    for plane in plane_dict.keys():
    
        with parallel_backend('dask',scheduler_host=cluster.scheduler._address,wait_for_workers_timeout=20):
            mean_int = Parallel(n_jobs=-1)(delayed(get_pixels)(lab, pl = plane) for lab in range(val))
        mean_intensity_per_marker.update({plane:mean_int})
    

    with open(Path(snakemake.output[0]), 'wb') as f:
        pickle.dump(mean_intensity_per_marker, f)
        
    client.close()
    cluster.close()
        
    
else:
    
    def get_mean_intensity(pl):
        result_ar = np.zeros(mx)
        tr = torch.from_numpy(pl)
        for r in range(mx):
            result_ar[r] = (tr[lab == r+1]).float().mean()
        return result_ar

    lab = torch.from_numpy(labels.astype('int'))
    mx = np.max(labels)
    
    mean_intensity_per_marker = {}
    for plane in plane_dict.keys():
        pl = plane_dict[plane].values
        mean_int = get_mean_intensity(pl)
        mean_intensity_per_marker.update({plane:mean_int})
    
    with open(Path(snakemake.output[0]), 'wb') as f:
        pickle.dump(mean_intensity_per_marker, f)
    
        

    

/gpfs/commons/home/jsingh/.conda/envs/spatial/lib/python3.9/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


In [3]:
mean_intensity_per_marker['ELAVL2']

[204.1565924369748,
 219.2435027124774,
 197.16119736842103,
 233.68440799999993,
 199.79664189189182,
 194.20469999999986,
 183.08972433460076,
 358.44147388059764,
 418.94018958333294,
 212.90533221476502,
 156.59556486486485,
 365.7232986111111,
 165.17839473684214,
 178.10893412526994,
 163.8208304347826,
 159.77677124183006,
 174.28523369565218,
 171.08142982456144,
 181.10694495412844,
 192.6503812351543,
 176.92490237226275,
 182.31477560975608,
 194.53574879227057,
 328.61229999999983,
 278.45205803571446,
 194.80943018480494,
 221.98814909638546,
 199.89550384615387,
 173.03966400000004,
 167.5411196172249,
 228.30494692005246,
 181.7110406504065,
 232.51638414634147,
 175.5658253968254,
 186.34737719298246,
 185.1118387096774,
 484.4968318584066,
 187.3802853982301,
 170.65271720116618,
 183.47643689320375,
 182.9458898809524,
 214.60364705882353,
 277.55134895833316,
 170.77353632478636,
 182.5046763485477,
 198.67561320754717,
 168.0955785791173,
 184.14165252293574,
 193.8

2022-08-25 13:57:55,627 - distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client
2022-08-25 13:57:55,629 - distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client
2022-08-25 13:57:55,629 - distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client
2022-08-25 13:57:55,634 - distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client
